In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import requests
from bs4 import BeautifulSoup
import pandas as pd
import itertools
import re
from tqdm import tqdm_notebook
import string
import numpy as np

# Удаление стоп слов и стемминг

In [45]:
df_final = pd.read_csv("coursera_clean.csv")
df_stack = pd.read_csv("tidy_questions.csv", index_col = "index", encoding = "utf-8", dtype={'index':np.str})

In [47]:
df_stack.drop("Unnamed: 0", axis = 1, inplace = True)

In [48]:
df_stack.dropna(inplace=True)

In [65]:
df_stack.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288227 entries, 23755195 to 31632133
Data columns (total 1 columns):
Text    288227 non-null object
dtypes: object(1)
memory usage: 4.4+ MB


In [4]:
df_final.head()

,index,about,level,rating,rating_count,syllabus
0,https://www.coursera.org/learn/python,this course aims to teach everyone the basics...,NaN,4.8,29620,chapter one why we program these are the cours...
1,https://www.coursera.org/learn/algorithms-part1,this course covers the essential information ...,Intermediate,4.9,2648,course introduction welcome to algorithms part...
2,https://www.coursera.org/learn/python-data-ana...,this course will introduce the learner to the...,Intermediate,4.5,3836,week in this week youl get an introduction to ...
3,https://www.coursera.org/learn/algorithmic-too...,the course covers basic algorithmic technique...,NaN,4.6,2355,welcome welcome to the first module of data st...
4,https://www.coursera.org/learn/python-data,this course will introduce the core data stru...,NaN,4.8,16116,chapter six strings in this class we pick up w...


In [6]:
stop_words = set(stopwords.words('english'))

In [7]:
def stemm_stop_words(string):
    string = str(string)
    tokens = word_tokenize(string)
    
    ps = PorterStemmer()

    filtered_sentence = [w for w in tokens if not w in stop_words]
    
    stem_sentence = [ps.stem(w) for w in filtered_sentence]
    
    string = " ".join(stem_sentence)
    return string

In [8]:
df_final["about"] = df_final["about"].apply(stemm_stop_words)
df_final["syllabus"] = df_final["syllabus"].apply(stemm_stop_words)

In [9]:
df_final.head()

,index,about,level,rating,rating_count,syllabus
0,https://www.coursera.org/learn/python,cours aim teach everyon basic program comput u...,NaN,4.8,29620,chapter one program coursewid materi well firs...
1,https://www.coursera.org/learn/algorithms-part1,cours cover essenti inform everi seriou progra...,Intermediate,4.9,2648,cours introduct welcom algorithm part video ex...
2,https://www.coursera.org/learn/python-data-ana...,cours introduc learner basic python program en...,Intermediate,4.5,3836,week week youl get introduct field data scienc...
3,https://www.coursera.org/learn/algorithmic-too...,cours cover basic algorithm techniqu idea comp...,NaN,4.6,2355,welcom welcom first modul data structur algori...
4,https://www.coursera.org/learn/python-data,cours introduc core data structur python progr...,NaN,4.8,16116,chapter six string class pick left previou cla...


## Запись в цсв

In [10]:
df_final.to_csv("coursera.csv")

# CNN модель

In [8]:
df_final = pd.read_csv('coursera.csv').drop('Unnamed: 0',axis=1)

In [14]:
df_train.head()

,Text
index,
https://www.coursera.org/learn/python,cours aim teach everyon basic program comput u...
https://www.coursera.org/learn/algorithms-part1,cours cover essenti inform everi seriou progra...
https://www.coursera.org/learn/python-data-analysis,cours introduc learner basic python program en...
https://www.coursera.org/learn/algorithmic-toolbox,cours cover basic algorithm techniqu idea comp...
https://www.coursera.org/learn/python-data,cours introduc core data structur python progr...


In [10]:
df_final['Text'] = df_final["about"] + df_final["syllabus"]
df_train = df_final[["index", "Text"]]

In [13]:
df_train.set_index('index', drop = True, inplace=True)

In [18]:
df_train.to_csv('coursera_union.csv')

In [51]:
df_cnn = pd.concat([df_train, df_stack], axis=0)

In [50]:
df_cnn.tail(10)

,Text
index,
24346956,pleas take look let know chang background colo...
33208055,could two php version differ path server curre...
14590341,easi get work run littl demo use googl homepag...
26497549,assign suppos creat rock paper scissor game us...
20965290,new facebook app develop found fb app develop ...
42507623,click button mobil chrome blue color appear se...
7167806,applic sign jar certif mine wan run applic web...
32203664,alreadi creat splash screen use jqueri fadeout...
9704650,know question seem imposs sure somebodi help d...


In [102]:
#Define the TFIDF vectorizer that will be used to process the data
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_cnn['Text'])
tfidf_courses = tfidf_vectorizer.transform(df_train['Text'])
tfidf_questions = tfidf_vectorizer.transform(df_stack['Text'])

In [93]:
tfidf_matrix.shape, tfidf_courses.shape, tfidf_questions.shape

((288760, 176039), (533, 176039), (288227, 176039))

In [94]:
nbrs = NearestNeighbors(n_neighbors=100).fit(tfidf_matrix)
nbrs_courses = NearestNeighbors(n_neighbors=100).fit(tfidf_courses)
nbrs_questions = NearestNeighbors(n_neighbors=100).fit(tfidf_questions)

In [77]:
nbrs

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=100, p=2, radius=1.0)

In [37]:
df_cnn.to_csv('coursera_ques.csv')

In [110]:
df_cnn.head()

,Text
index,
https://www.coursera.org/learn/python,cours aim teach everyon basic program comput u...
https://www.coursera.org/learn/algorithms-part1,cours cover essenti inform everi seriou progra...
https://www.coursera.org/learn/python-data-analysis,cours introduc learner basic python program en...
https://www.coursera.org/learn/algorithmic-toolbox,cours cover basic algorithm techniqu idea comp...
https://www.coursera.org/learn/python-data,cours introduc core data structur python progr...


In [103]:
def get_closest_neighs(name, target='course'):
    if target == 'course':
        nbrs_local = nbrs_courses
    elif target == 'question':
        nbrs_local = nbrs_questions
    elif target == 'multi':
        nbrs_local = nbrs
    row = df_cnn.index.get_loc(name)
    distances, indices = nbrs_local.kneighbors(tfidf_matrix.getrow(row))
    indexes_similar = pd.Series(indices.flatten()).map(df_cnn.reset_index()['index'])
    result = pd.DataFrame({'distance': distances.flatten(), 'index': indexes_similar})
    
    return result

In [114]:
pred = get_closest_neighs("https://www.coursera.org/learn/python", "multi")

In [112]:
print(pred.loc[0, "index"])

45578306


In [115]:
pred

,distance,index
0,0.000000,https://www.coursera.org/learn/python
1,0.637485,https://www.coursera.org/learn/python-data
2,0.659529,https://www.coursera.org/learn/python-network-...
3,0.797718,https://www.coursera.org/learn/python-databases
4,0.818484,https://www.coursera.org/learn/python-programming
5,0.867314,https://www.coursera.org/learn/programming-lan...
6,0.878428,https://www.coursera.org/learn/python-data-pro...
7,0.880894,https://www.coursera.org/learn/objective-c
8,0.886610,https://www.coursera.org/learn/python-represen...
9,0.892057,https://www.coursera.org/learn/arduino
